# UNet 3D Implementation in Keras


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nibabel as nib
import glob

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
root_dir = "/content/drive/My Drive/"

project_folder = "FYP/MONAI_Lung/"
data_loc = "FYP/MONAI_Lung/Data/Task06_Lung/"

data_dir = root_dir + data_loc


def create_and_set_working_directory(project_folder):
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  os.chdir(root_dir + project_folder)
  print('Your working directory was changed to ' + root_dir + project_folder)

create_and_set_working_directory(project_folder)

Mounted at /content/drive
Your working directory was changed to /content/drive/My Drive/FYP/MONAI_Lung/


In [ ]:
train_images = sorted(
    glob.glob(os.path.join(data_dir, "imagesTr", "*.nii.gz")))
train_labels = sorted(
    glob.glob(os.path.join(data_dir, "labelsTr", "*.nii.gz")))
data_dicts = [
    {"image": image_name, "label": label_name}
    for image_name, label_name in zip(train_images, train_labels)
]
train_files = data_dicts

In [ ]:
len(train_files)

63

In [ ]:
total_sections = 0

for i in range(len(train_files)):
  mri_array = nib.load(train_files[i]['label'])
  slices = mri_array.shape[2]
  sections = int(float(slices)/64.0)+1
  total_sections += sections
print('total number of sections: ', total_sections)

total number of sections:  302


In [ ]:
from skimage.measure import block_reduce
def make_block_reduce( input_layers, dim=(2,2,2), mode=np.mean ):
    stacked_layers = [ block_reduce( image, dim, mode ) for image in input_layers ]
    return np.asarray( stacked_layers, dtype='float32' )

In [ ]:
# finding total number of images in a given number of MRI files
# counter = 0
# for i in range(len(train_files)):
#   mri = nib.load(train_files[i]['label'])
#   counter += mri.shape[2]

# print(counter)

In [ ]:
# mri = nib.load(train_files[0]['image'])
# test_data_array = mri.get_fdata()
# print(test_data_array.shape)

# data_arranged_1 = np.swapaxes(test_data_array,0,2)
# data_arranged_2 = np.swapaxes(data_arranged_1,1,2)

# test_data_array = make_block_reduce(data_arranged_2, (2,2), np.mean)
# test_slice = test_data_array[150, :, :]
# plt.imshow(test_slice.T, cmap='gray', origin='lower')
# plt.xlabel('First axis')
# plt.ylabel('Second axis')
# plt.colorbar(label='Signal intensity')
# plt.show()

# test_data_array = make_block_reduce(test_data_array, (2,2), np.mean)
# test_slice = test_data_array[150, :, :]
# plt.imshow(test_slice.T, cmap='gray', origin='lower')
# plt.xlabel('First axis')
# plt.ylabel('Second axis')
# plt.colorbar(label='Signal intensity')
# plt.show()

# test_data_array = make_block_reduce(test_data_array, (2,2), np.mean)
# test_slice = test_data_array[150, :, :]
# plt.imshow(test_slice.T, cmap='gray', origin='lower')
# plt.xlabel('First axis')
# plt.ylabel('Second axis')
# plt.colorbar(label='Signal intensity')
# plt.show()

# test_data_array = make_block_reduce(test_data_array, (2,2), np.mean)
# test_slice = test_data_array[150, :, :]
# plt.imshow(test_slice.T, cmap='gray', origin='lower')
# plt.xlabel('First axis')
# plt.ylabel('Second axis')
# plt.colorbar(label='Signal intensity')
# plt.show()

# test_data_array = make_block_reduce(test_data_array, (2,2), np.mean)
# test_slice = test_data_array[150, :, :]
# plt.imshow(test_slice.T, cmap='gray', origin='lower')
# plt.xlabel('First axis')
# plt.ylabel('Second axis')
# plt.colorbar(label='Signal intensity')
# plt.show()

# test_data_array = make_block_reduce(test_data_array, (2,2), np.mean)
# test_slice = test_data_array[150, :, :]
# plt.imshow(test_slice.T, cmap='gray', origin='lower')
# plt.xlabel('First axis')
# plt.ylabel('Second axis')
# plt.colorbar(label='Signal intensity')
# plt.show()

# test_data_array = make_block_reduce(test_data_array, (2,2), np.mean)
# test_slice = test_data_array[150, :, :]
# plt.imshow(test_slice.T, cmap='gray', origin='lower')
# plt.xlabel('First axis')
# plt.ylabel('Second axis')
# plt.colorbar(label='Signal intensity')
# plt.show()

# test_data_array = make_block_reduce(test_data_array, (2,2), np.mean)
# test_slice = test_data_array[150, :, :]
# plt.imshow(test_slice.T, cmap='gray', origin='lower')
# plt.xlabel('First axis')
# plt.ylabel('Second axis')
# plt.colorbar(label='Signal intensity')
# plt.show()

label_data_array = np.empty((302,64,256,256))

counter=0

for i in range(len(train_files)):
  mri_array = nib.load(train_files[i]['label']).get_fdata()
  data_arranged_1 = np.swapaxes(mri_array,0,2)
  data_arranged_2 = np.swapaxes(data_arranged_1,1,2)
  mri_array = make_block_reduce(data_arranged_2, (2,2), np.mean) # shape = (X,256,256)
  for j in range(int(float(mri_array.shape[0])/64.0) +1):
    if j < (int(float(mri_array.shape[0])/64.0)):
      start_index = 64*j
      end_index = 64*(j+1)
    else:
      end_index = mri_array.shape[0]
      start_index = end_index-64

    label_data_array[counter,:,:,:] = mri_array[start_index:end_index,:,:]
    counter += 1
print("label data array shape: ", label_data_array.shape)

label data array shape:  (302, 64, 256, 256)


In [ ]:
output_layers = label_data_array.reshape(label_data_array.shape + (1,))

In [ ]:
# now get the input data 

image_data_array = np.empty((302,64,256,256))

counter=0

for i in range(len(train_files)):
  mri_array = nib.load(train_files[i]['image']).get_fdata()
  data_arranged_1 = np.swapaxes(mri_array,0,2)
  data_arranged_2 = np.swapaxes(data_arranged_1,1,2)
  mri_array = make_block_reduce(data_arranged_2, (2,2), np.mean) # shape = (X,256,256)
  for j in range(int(float(mri_array.shape[0])/64.0) +1):
    if j < (int(float(mri_array.shape[0])/64.0)):
      start_index = 64*j
      end_index = 64*(j+1)
    else:
      end_index = mri_array.shape[0]
      start_index = end_index-64

    image_data_array[counter,:,:,:] = mri_array[start_index:end_index,:,:]
    counter += 1

print("image data array shape: ", image_data_array.shape)

image data array shape:  (302, 64, 256, 256)


In [ ]:
counter

302

In [ ]:
image_data_array = image_data_array .reshape( image_data_array.shape + (1,) )

In [ ]:
print("image shapes expressed as: (image number, x dimension, y dimension, channels)")
print("input images: ", image_data_array.shape)

print("output labels resolution: ", output_layers.shape)

image shapes expressed as: (image number, x dimension, y dimension, channels)
input images:  (302, 64, 256, 256, 1)
output labels resolution:  (302, 64, 256, 256, 1)


In [ ]:
%tensorflow_version 1.x
import tensorflow
print( tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.initializers import he_normal
kr = regularizers.l2( 1.0e-3 )
ki = he_normal()

In [ ]:
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import Conv3D, Conv3DTranspose
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers.merge import add

init = Input(shape=(64, 256, 256, 1))
init_ = Conv3D( 32, kernel_size=(17, 17, 5), activation = "relu", strides = 1, padding="same", kernel_regularizer=kr, kernel_initializer=ki )( init )
l1 = Conv3D( 64, kernel_size=(5, 5, 3), activation = "relu", strides = 2, padding="same", kernel_regularizer=kr, kernel_initializer=ki )( init_ )
l2=Conv3D(128, kernel_size=(3, 3, 3), activation="relu", strides=2, padding='same', kernel_regularizer=kr, kernel_initializer=ki)(l1) 
l3=Conv3D(192, kernel_size=(3, 3, 3), activation="relu", strides=2, padding='same', kernel_regularizer=kr, kernel_initializer=ki)(l2) 
l4=Conv3D(256, kernel_size=(3, 3, 3), activation="relu", strides=2, padding='same', kernel_regularizer=kr, kernel_initializer=ki)(l3) 
l5 = Conv3D(384, kernel_size=(3, 3, 3), activation="relu", strides=2, padding='same', kernel_regularizer=kr, kernel_initializer=ki)(l4) 

l13=Conv3DTranspose(256, kernel_size=(3, 3, 3),  activation="relu", strides=2, padding='same', kernel_regularizer=kr, kernel_initializer=ki)(l5) 
l15=add([l13, l4]) 
l16=Conv3DTranspose(192, kernel_size=(3, 3, 3), strides=2, activation="relu", padding='same', kernel_regularizer=kr, kernel_initializer=ki)(l15) 
l18=add([l16, l3]) 
l19=Conv3DTranspose(128, kernel_size=(3, 3, 3), activation="relu", strides=2, padding='same', kernel_regularizer=kr, kernel_initializer=ki)(l18) 
l21=add([l19, l2]) 
l22=Conv3DTranspose(64, kernel_size=(3, 3, 3), activation="relu", strides=2, padding='same', kernel_regularizer=kr, kernel_initializer=ki)(l21) 
l24=add([l22, l1])
l25=Conv3DTranspose(32, kernel_size=(3, 3, 3), activation="relu", strides=2, padding='same', kernel_regularizer=kr, kernel_initializer=ki)(l24) 
l27=add([l25, init_])
llast=Conv3D(16 , kernel_size=(3, 3, 3), activation='relu', strides=1, padding='same', kernel_regularizer=kr, kernel_initializer=ki)(l27)
llast=Conv3D(8 , kernel_size=(3, 3, 3), activation='relu', strides=1, padding='same', kernel_regularizer=kr, kernel_initializer=ki)(llast)
last_output=Conv3D(1, kernel_size=(17, 17, 5), activation='sigmoid', strides=1, padding='same', kernel_regularizer=kr, kernel_initializer=ki, name="gen_256")(llast)
unet = Model(init, last_output)

In [ ]:
unet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 256, 256 0                                            
__________________________________________________________________________________________________
conv3d_8 (Conv3D)               (None, 64, 256, 256, 46272       input_2[0][0]                    
__________________________________________________________________________________________________
conv3d_9 (Conv3D)               (None, 32, 128, 128, 153664      conv3d_8[0][0]                   
__________________________________________________________________________________________________
conv3d_10 (Conv3D)              (None, 16, 64, 64, 1 221312      conv3d_9[0][0]                   
____________________________________________________________________________________________

In [ ]:
sess = tensorflow.Session(config=tensorflow.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



In [ ]:
unet.compile( loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

In [ ]:
model_name = 'UNet_3D_ver_1/' #CHANGE

def create_directory(dir):
  if os.path.isdir(dir) == False:
    os.mkdir(dir)
    print(dir + ' did not exist but was created.')
  else:
    print(dir + ' already exists. Double ccheck another model will not be overwritten!')

create_directory(root_dir + project_folder + model_name)

/content/drive/My Drive/FYP/MONAI_Lung/UNet_3D_ver_1/ already exists. Double ccheck another model will not be overwritten!


In [ ]:
from keras.callbacks import CSVLogger

csv_filepath = 'csv_log.csv'
csv_dir = root_dir + project_folder + model_name + csv_filepath

csv_logger = CSVLogger(csv_dir, append=True, separator=';')

In [ ]:
# IF RESUMING, UNCOMMENT BELOW:

# mcnn.load_weights(checkpoint_dir)

In [ ]:
history = unet.fit(image_data_array, output_layers, batch_size=1, epochs=40, verbose=1,validation_split=0.25, callbacks=[csv_logger])

Train on 226 samples, validate on 76 samples
Epoch 1/40
226/226 [==============================] - 587s 3s/sample - loss: 3.0782 - acc: 0.9888 - val_loss: 1.3556 - val_acc: 0.9895
Epoch 2/40
226/226 [==============================] - 564s 2s/sample - loss: 1.2435 - acc: 0.9932 - val_loss: 0.9450 - val_acc: 0.9943
Epoch 3/40
226/226 [==============================] - 564s 2s/sample - loss: 0.8872 - acc: 0.9957 - val_loss: 0.8226 - val_acc: 0.9963
Epoch 4/40
226/226 [==============================] - 564s 2s/sample - loss: 0.7604 - acc: 0.9962 - val_loss: 0.6618 - val_acc: 0.9966
Epoch 5/40
226/226 [==============================] - 564s 2s/sample - loss: 0.6487 - acc: 0.9973 - val_loss: 0.6039 - val_acc: 0.9988
Epoch 6/40
226/226 [==============================] - 564s 2s/sample - loss: 0.6991 - acc: 0.9983 - val_loss: 0.5777 - val_acc: 0.9974
Epoch 7/40
226/226 [==============================] - 564s 2s/sample - loss: 0.5100 - acc: 0.9982 - val_loss: 0.4476 - val_acc: 0.9989
Epoch 8/40

In [ ]:
unet.save_weights('./UNet_3D_ver_1.h5', overwrite=True)